In [1]:
import findspark
# findspark.init('C:/BigDataLocalSetup/spark')

import pyspark
from pyspark import SparkContext
from pyspark.sql.functions import *
from pyspark.sql import SparkSession, SQLContext

ModuleNotFoundError: No module named 'findspark'

In [2]:
spark = SparkSession.\
        builder.\
        appName("pyspark-notebook2").\
        config("spark.executor.memory", "1g").\
        config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
        getOrCreate()

In [3]:
#findspark.add_packages('org.mongodb.spark:mongo-spark-connector_2.12:3.0.0')

# Get data from mongoDB

In [4]:
df_manufacturer = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.MANUFACTURER").load()
df_distributor = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.DISTRIBUTOR").load()
df_hospital = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.HOSPITAL").load()
df_user_roles = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.USER_ROLES").load()
df_product = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()
df_correlation_data = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.CORRELATION_DATA").load()
df_gpo = spark.read.format("mongo").option("uri",
                                "mongodb://127.0.0.1:27017/MED_EX_CHAIN.GPO").load()

NameError: name 'spark' is not defined

In [ ]:
df_manufacturer.select(col('MANUFACTURER_ID'), col('VERSION')).show(20, False)

In [ ]:
df_manufacturer.createOrReplaceTempView('manufacturer_view')
df_distributor.createOrReplaceTempView('distributor_view')
df_hospital.createOrReplaceTempView('hospital_view')
df_user_roles.createOrReplaceTempView('user_view')
df_gpo.createOrReplaceTempView('gpo_view')
df_product.createOrReplaceTempView('product_view')

In [ ]:
df_manufacturer=spark.sql('select * from manufacturer_view where (MANUFACTURER_ID,VERSION) in \
(select MANUFACTURER_ID, max(VERSION) from manufacturer_view group by MANUFACTURER_ID)')

df_distributor=spark.sql('select * from distributor_view where (DISTRIBUTOR_ID,VERSION) in \
(select DISTRIBUTOR_ID, max(VERSION) from distributor_view group by DISTRIBUTOR_ID)')

df_hospital=spark.sql('select * from hospital_view where (HOSPITAL_ID,VERSION) in \
(select HOSPITAL_ID, max(VERSION) from hospital_view group by HOSPITAL_ID)')

df_user_roles=spark.sql('select * from user_view where (USER_ID,VERSION) in \
(select USER_ID, max(VERSION) from user_view group by USER_ID)')

df_gpo=spark.sql('select * from gpo_view where (GPO_ID,VERSION) in \
(select GPO_ID, max(VERSION) from gpo_view group by GPO_ID)')

df_product=spark.sql('select * from product_view where (PRODUCT_ID,VERSION) in \
(select PRODUCT_ID, max(VERSION) from product_view group by PRODUCT_ID)')

In [ ]:
'''#df_product = spark.read.format("mongo").option("uri",
 #                               "mongodb://127.0.0.1:27017/MED_EX_CHAIN.PRODUCT").load()


product_data = pd.DataFrame(list(product.find()))
#cor =dict(product_data[['PRODUCT_ID', 'CORRELATION']].values)

#print([i['DISTRIBUTOR_SKU'] for i in cor[2001.0] if 'DISTRIBUTOR_SKU' in i])
product_data.columns'''

In [ ]:
'''distributor_name=[]
distributor_sku=[]
hospital_name=[]
hospital_sku=[]
for i in cor:
    for j in cor[i]:
        if 'DISTRIBUTOR_NAME' in j:
            distributor_name.append(j['DISTRIBUTOR_NAME'])
            distributor_sku.append(j['DISTRIBUTOR_SKU'])
        if 'HOSPITAL_NAME' in j:
            hospital_name.append(j['HOSPITAL_NAME'])
            hospital_sku.append(j['HOSPITAL_SKU'])'''

In [ ]:
'''df_correlation = pd.DataFrame({'PRODUCT_ID':list(cor.keys()),
                   'DISTRIBUTOR_SKU':distributor_sku,
                  'DISTRIBUTOR_NAME':distributor_name,
                  'HOSPITAL_NAME':hospital_name,
                  'HOSPITAL_SKU':hospital_sku,
                        'MANUFACTURER_NAME':product_data['MANUFACTURER']})'''

In [ ]:
'''df_correlation_data.show(20,False)'''

# Create nodes in Neo4J

In [ ]:
from py2neo import *

graph = Graph("bolt://localhost:7687", auth=("neo4j", "karan"))

In [ ]:
'''# create product node
for i in range(0,len(product_data)):
    graph.run("create (n:Product{product_id:$product_id,\
                                 product_category:$product_category,\
                                 base_price:$base_price,\
                                 color:$color,\
                                 capacity:$capacity,\
                                 manufacturer:$manufacturer,\
                                 description:$description,\
                                 mfg_date:$mfg_date,\
                                 expiry_date:$expiry_date,\
                                 is_pack:$is_pack,\
                                 product_status:$product_status,\
                                 product_type_id:$product_type_id,\
                                 size:$size,\
                                 short_description:$short_description,\
                                 units_in_pack:$units_in_pack,\
                                 uom:$uom,\
                                 volume:$volume,\
                                 weight_uom:$weight_uom,\
                                 product_image:$product_image})",
           product_id=product_data['PRODUCT_ID'][i],product_category=product_data['PRODUCT_CATEGORY'][i],
           name=product_data['NAME'][i],manufacturer=product_data['MANUFACTURER'][i],
             base_price=product_data['BASE_PRICE'][i],color=product_data['COLOR'][i],
             capacity=product_data['Capacity'][i],description=product_data['DESCRIPTION'][i],
             mfg_date=product_data['MFG_DATE'][i],expiry_date=product_data['EXPIRY_DATE'][i],
             is_pack=product_data['IS_PACK'][i],product_status=product_data['PRODUCT_STATUS'][i],
             product_type_id=product_data['PRODUCT_TYPE_ID'][i],size=product_data['SIZE'][i],
             short_description=product_data['SHORT_DESCRIPTION'][i],
             units_in_pack=product_data['UNITS_IN_PACK'][i],
             uom=product_data['UOM'][i],volume=product_data['VOLUME'][i],
             weight_uom=product_data['WEIGHT_UOM'][i],product_image=product_data['PRODUCT_IMAGE'][i])'''

In [ ]:
df_product.columns

In [ ]:
for record in df_product.rdd.collect():
    graph.run("merge (p:Product{product_id:$product_id})\
              on create set \
                                 p.product_category=$product_category,\
                                 p.base_price=$base_price,\
                                 p.name=$name,\
                                 p.color=$color,\
                                 p.manufacturer=$manufacturer,\
                                 p.description=$description,\
                                 p.mfg_date=$mfg_date,\
                                 p.expiry_date=$expiry_date,\
                                 p.is_pack=$is_pack,\
                                 p.product_status=$product_status,\
                                 p.product_type_id=$product_type_id,\
                                 p.size=$size,\
                                 p.short_description=$short_description,\
                                 p.units_in_pack=$units_in_pack,\
                                 p.uom=$uom,\
                                 p.volume=$volume,\
                                 p.weight_uom=$weight_uom,\
                                 p.product_image=$product_image,\
                                 p.start_date=$start_date,\
                                 p.end_date=$end_date,\
                                 p.last_update_date=$last_update_date,\
                                 p.version=$version\
                on match set\
                                 p.product_category=$product_category,\
                                 p.base_price=$base_price,\
                                 p.name=$name,\
                                 p.color=$color,\
                                 p.manufacturer=$manufacturer,\
                                 p.description=$description,\
                                 p.mfg_date=$mfg_date,\
                                 p.expiry_date=$expiry_date,\
                                 p.is_pack=$is_pack,\
                                 p.product_status=$product_status,\
                                 p.product_type_id=$product_type_id,\
                                 p.size=$size,\
                                 p.short_description=$short_description,\
                                 p.units_in_pack=$units_in_pack,\
                                 p.uom=$uom,\
                                 p.volume=$volume,\
                                 p.weight_uom=$weight_uom,\
                                 p.product_image=$product_image,\
                                 p.start_date=$start_date,\
                                 p.end_date=$end_date,\
                                 p.last_update_date=$last_update_date,\
                                 p.version=$version",
             product_id=record['PRODUCT_ID'],product_category=record['PRODUCT_CATEGORY'],
             base_price=record['BASE_PRICE'],name=record['NAME'],
             color=record['COLOR'],manufacturer=record['MANUFACTURER'],
             description=record['DESCRIPTION'],mfg_date=record['MFG_DATE'],
             expiry_date=record['EXPIRY_DATE'],
             is_pack=record['IS_PACK'],product_status=record['PRODUCT_STATUS'],
             product_type_id=record['PRODUCT_TYPE_ID'],size=record['SIZE'],
             short_description=record['SHORT_DESCRIPTION'],units_in_pack=record['UNITS_IN_PACK'],
             uom=record['UOM'],volume=record['VOLUME'],
             weight_uom=record['WEIGHT_UOM'],product_image=record['PRODUCT_IMAGE'],
             start_date=record['START_DATE'],
             end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
             version=record['VERSION'])

In [ ]:
'''# create Manufacturer node
for record in df_manufacturer.rdd.collect():
    graph.run("merge (n:Manufacturer{manufacturer_id:$manufacturer_id,\
                                      country_of_origin:$country_of_origin,\
                                      manufacturer_name:$manufacturer_name,\
                                      manufacturer_address:$manufacturer_address,\
                                      manufacturer_org_type:$manufacturer_org_type,\
                                      manufacturer_short_name:$manufacturer_short_name,\
                                      source_identifier:$source_identifier,\
                                      start_date:$start_date,\
                                      end_date:$end_date,\
                                      last_update_date:$last_update_date,\
                                      last_update_user:$last_update_user,\
                                      version_number:$version_number})",
        manufacturer_id=record['MANUFACTURER_ID'],manufacturer_name=record['MANUFACTURER_NAME'],
        country_of_origin=record['COUNTRY_OF_ORIGIN'],manufacturer_address=record['MANUFACTURER_ADDRESS'],
        manufacturer_org_type=record['MANUFACTURER_ORG_TYPE'],
        manufacturer_short_name=record['MANUFACTURER_SHORT_NAME'],
        source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
        end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
        last_update_user=record['LAST_UPDATE_USER'],version_number=record['VERSION'],
        start_date=record['START_DATE'],end_date=record['END_DATE'],
        last_update_date=record['LAST_UPDATE_DT'],
        last_update_user=record['LAST_UPDATE_USER'],version=record['VERSION'])
'''

In [ ]:
# create Manufacturer node
for record in df_manufacturer.rdd.collect():
    graph.run("merge (m:Manufacturer{manufacturer_id:$manufacturer_id})\
               on create set\
                                      m.country_of_origin=$country_of_origin,\
                                      m.manufacturer_name=$manufacturer_name,\
                                      m.manufacturer_address=$manufacturer_address,\
                                      m.manufacturer_org_type=$manufacturer_org_type,\
                                      m.manufacturer_short_name=$manufacturer_short_name,\
                                      m.source_identifier=$source_identifier,\
                                      m.start_date=$start_date,\
                                      m.end_date=$end_date,\
                                      m.last_update_date=$last_update_date,\
                                      m.last_update_user=$last_update_user,\
                                      m.version=$version\
                on match set\
                                      m.country_of_origin=$country_of_origin,\
                                      m.manufacturer_name=$manufacturer_name,\
                                      m.manufacturer_address=$manufacturer_address,\
                                      m.manufacturer_org_type=$manufacturer_org_type,\
                                      m.manufacturer_short_name=$manufacturer_short_name,\
                                      m.source_identifier=$source_identifier,\
                                      m.start_date=$start_date,\
                                      m.end_date=$end_date,\
                                      m.last_update_date=$last_update_date,\
                                      m.last_update_user=$last_update_user,\
                                      m.version=$version",
        manufacturer_id=record['MANUFACTURER_ID'],manufacturer_name=record['MANUFACTURER_NAME'],
        country_of_origin=record['COUNTRY_OF_ORIGIN'],manufacturer_address=record['MANUFACTURER_ADDRESS'],
        manufacturer_org_type=record['MANUFACTURER_ORG_TYPE'],
        manufacturer_short_name=record['MANUFACTURER_SHORT_NAME'],
        source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
        end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
        last_update_user=record['LAST_UPDATE_USER'],version=record['VERSION'])


In [ ]:
# create Distributor node
for record in df_distributor.rdd.collect():
    graph.run("merge (d:Distributor{distributor_id:$distributor_id})\
                on create set\
                                     d.country_of_origin=$country_of_origin,\
                                     d.distributor_name=$distributor_name,\
                                     d.distributor_org_type=$distributor_org_type,\
                                     d.distributor_address=$distributor_address,\
                                     d.distributor_short_name=$distributor_short_name,\
                                     d.source_identifier=$source_identifier,\
                                     d.start_date=$start_date,\
                                     d.end_date=$end_date,\
                                     d.last_update_date=$last_update_date,\
                                     d.last_update_user=$last_update_user,\
                                     d.version=$version\
                on match set\
                                     d.country_of_origin=$country_of_origin,\
                                     d.distributor_name=$distributor_name,\
                                     d.distributor_org_type=$distributor_org_type,\
                                     d.distributor_address=$distributor_address,\
                                     d.distributor_short_name=$distributor_short_name,\
                                     d.source_identifier=$source_identifier,\
                                     d.start_date=$start_date,\
                                     d.end_date=$end_date,\
                                     d.last_update_date=$last_update_date,\
                                     d.last_update_user=$last_update_user,\
                                     d.version=$version",
         distributor_id=record['DISTRIBUTOR_ID'],distributor_name=record['DISTRIBUTOR_NAME'],
         country_of_origin=record['COUNTRY_OF_ORIGIN'],distributor_org_type=record['DISTRIBUTOR_ORG_TYPE'],
distributor_address=record['DISTRIBUTOR_ADDRESS'],distributor_short_name=record['DISTRIBUTOR_SHORT_NAME'],
         source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
         end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
         last_update_user=record['LAST_UPDATE_USER'],version=record['VERSION'])


In [ ]:
'''# create Distributor node
for record in df_distributor.rdd.collect():
    graph.run("create (n:Distributor{distributor_id:$distributor_id, \
                                     country_of_origin:$country_of_origin,\
                                     distributor_name:$distributor_name,\
                                     distributor_org_type:$distributor_org_type,\
                                     distributor_address:$distributor_address,\
                                     distributor_short_name:$distributor_short_name,\
                                     source_identifier:$source_identifier,\
                                     start_date:$start_date,\
                                     end_date:$end_date,\
                                     last_update_date:$last_update_date,\
                                     last_update_user:$last_update_user,\
                                     version_number:$version_number})",
         distributor_id=record['DISTRIBUTOR_ID'],distributor_name=record['DISTRIBUTOR_NAME'],
         country_of_origin=record['COUNTRY_OF_ORIGIN'],distributor_org_type=record['DISTRIBUTOR_ORG_TYPE'],
distributor_address=record['DISTRIBUTOR_ADDRESS'],distributor_short_name=record['DISTRIBUTOR_SHORT_NAME'],
         source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
         end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
         last_update_user=record['LAST_UPDATE_USER'],version_number=record['VERSION_NUMBER'])
'''

In [ ]:
'''# create Hospital node
for record in df_hospital.rdd.collect():
    graph.run("merge (n:Hospital{hospital_id:$hospital_id,\
                                  hospital_address:$hospital_address,\
                                  country_of_origin:$country_of_origin,\
                                  hospital_name:$hospital_name, \
                                  hospital_short_name:$hospital_short_name,\
                                  hospital_type:$hospital_type,\
                                  source_identifier:$source_identifier,\
                                  start_date:$start_date,\
                                  end_date:$end_date,\
                                  last_update_date:$last_update_date,\
                                  last_update_user:$last_update_user,\
                                  version_number:$version_number})",
            hospital_id=record['HOSPITAL_ID'],hospital_name=record['HOSPITAL_NAME'],
            country_of_origin=record['COUNTRY_OF_ORIGIN'],hospital_type=record['HOSPITAL_TYPE'],
            hospital_short_name=record['HOSPITAL_SHORT_NAME'],hospital_address=record['HOSPITAL_ADDRESS'],
            source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
            end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
            last_update_user=record['LAST_UPDATE_USER'],version_number=record['VERSION_NUMBER'])'''

In [ ]:
# create Hospital node
for record in df_hospital.rdd.collect():
    graph.run("merge (h:Hospital{hospital_id:$hospital_id})\
                on create set\
                                  h.hospital_address=$hospital_address,\
                                  h.country_of_origin=$country_of_origin,\
                                  h.hospital_name=$hospital_name, \
                                  h.hospital_short_name=$hospital_short_name,\
                                  h.hospital_type=$hospital_type,\
                                  h.source_identifier=$source_identifier,\
                                  h.start_date=$start_date,\
                                  h.end_date=$end_date,\
                                  h.last_update_date=$last_update_date,\
                                  h.last_update_user=$last_update_user,\
                                  h.version=$version\
                on match set\
                                  h.hospital_address=$hospital_address,\
                                  h.country_of_origin=$country_of_origin,\
                                  h.hospital_name=$hospital_name, \
                                  h.hospital_short_name=$hospital_short_name,\
                                  h.hospital_type=$hospital_type,\
                                  h.source_identifier=$source_identifier,\
                                  h.start_date=$start_date,\
                                  h.end_date=$end_date,\
                                  h.last_update_date=$last_update_date,\
                                  h.last_update_user=$last_update_user,\
                                  h.version=$version",
            hospital_id=record['HOSPITAL_ID'],hospital_name=record['HOSPITAL_NAME'],
            country_of_origin=record['COUNTRY_OF_ORIGIN'],hospital_type=record['HOSPITAL_TYPE'],
            hospital_short_name=record['HOSPITAL_SHORT_NAME'],hospital_address=record['HOSPITAL_ADDRESS'],
            source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
            end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
            last_update_user=record['LAST_UPDATE_USER'],version=record['VERSION'])

In [ ]:
df_user_roles.columns

In [ ]:
# create User Role node
for record in df_user_roles.rdd.collect():
    graph.run("merge (u:User_Roles{user_id:$user_id})\
                on create set\
                                    u.country_of_origin=$country_of_origin,\
                                    u.company_name=$company_name,\
                                    u.designation=$designation,\
                                    u.email_id=$email_id,\
                                    u.first_name=$first_name,\
                                    u.last_name=$last_name,\
                                    u.password=$password,\
                                    u.phone_number=$phone_number,\
                                    u.user_address=$user_address,\
                                    u.user_role=$user_role,\
                                    u.start_date=$start_date,\
                                    u.end_date=$end_date,\
                                    u.last_update_date=$last_update_date,\
                                    u.version=$version\
              on match set\
                                    u.country_of_origin=$country_of_origin,\
                                    u.company_name=$company_name,\
                                    u.designation=$designation,\
                                    u.email_id=$email_id,\
                                    u.first_name=$first_name,\
                                    u.last_name=$last_name,\
                                    u.password=$password,\
                                    u.phone_number=$phone_number,\
                                    u.user_address=$user_address,\
                                    u.user_role=$user_role,\
                                    u.start_date=$start_date,\
                                    u.end_date=$end_date,\
                                    u.last_update_date=$last_update_date,\
                                    u.version=$version",
             company_name=record['COMPANY_NAME'],country_of_origin=record['COUNTRY_OF_ORIGIN'],
             designation=record['DESIGNATION'],
             email_id=record['EMAIL_ID'],first_name=record['FIRST_NAME'],
             last_name=record['LAST_NAME'],
             password=record['PASSWORD'],phone_number=record['PHONE_NUMBER'],
             user_address=record['USER_ADDRESS'],user_id=record['USER_ID'],
             user_role=record['USER_ROLE'],start_date=record['START_DATE'],
             end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
             version=record['VERSION'])

In [ ]:
# create correlation node
'''for i in range(0,len(df_correlation)):
    graph.run("merge (n:Correlation_Data{product_id:$product_id,\
                                          distributor_name:$distributor_name,\
                                          distributor_sku:$distributor_sku, \
                                          hospital_name:$hospital_name, \
                                          hospital_sku:$hospital_sku,\
                                          manufacturer_name:$manufacturer_name})",
                distributor_name=df_correlation['DISTRIBUTOR_NAME'][i],
                distributor_sku=df_correlation['DISTRIBUTOR_SKU'][i],
                hospital_name=df_correlation['HOSPITAL_NAME'][i],
                hospital_sku=df_correlation['HOSPITAL_SKU'][i],
                product_id=df_correlation['PRODUCT_ID'][i],
                manufacturer_name=df_correlation['MANUFACTURER_NAME'][i])'''

for record in df_correlation_data.rdd.collect():
    graph.run("merge (n:Correlation_Data{product_id:$product_id,\
                                          distributor_name:$distributor_name,\
                                          distributor_sku:$distributor_sku, \
                                          hospital_name:$hospital_name, \
                                          hospital_sku:$hospital_sku,\
                                          manufacturer_name:$manufacturer_name,\
                                          contract_number:$contract_number,\
                                          contract_price:$contract_price,\
                                          base_price:$base_price})",
                distributor_name=record['DISTRIBUTOR_NAME'],
                distributor_sku=record['DISTRIBUTOR_SKU'],
                hospital_name=record['HOSPITAL_NAME'],
                hospital_sku=record['HOSPITAL_SKU'],
                product_id=record['PRODUCT_ID'],
                manufacturer_name=record['MANUFACTURER_NAME'],
                contract_number=record['CONTRACT_NUMBER'],
                contract_price=record['CONTRACT_PRICE'],
                base_price=record['BASE_PRICE'])

In [ ]:
'''#create GPO node
for record in df_gpo.rdd.collect():
    graph.run("merge (n:GPO{gpo_id:$gpo_id,\
                            gpo_name:$gpo_name,\
                            gpo_short_name:$gpo_short_name, \
                            gpo_org_type:$gpo_org_type, \
                            gpo_address:$gpo_address,\
                            country_of_origin:$country_of_origin,\
                            source_identifier:$source_identifier,\
                            start_date:$start_date,\
                            end_date:$end_date,\
                            last_update_date:$last_update_date,\
                            last_update_user:$last_update_user,\
                            version_number:$version_number})",
                gpo_id=record['GPO_ID'],
                gpo_name=record['GPO_NAME'],
                gpo_short_name=record['GPO_SHORT_NAME'],country_of_origin=record['COUNTRY_OF_ORIGIN'],
                gpo_org_type=record['GPO_ORG_TYPE'],gpo_address=['GPO_ADDRESS'],
                source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
            end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
            last_update_user=record['LAST_UPDATE_USER'],version_number=record['VERSION_NUMBER'])'''

In [ ]:
df_gpo.columns

In [ ]:
#create GPO node
for record in df_gpo.rdd.collect():
    graph.run("merge (g:GPO{gpo_id:$gpo_id})\
              on create set\
                            g.gpo_name=$gpo_name,\
                            g.gpo_short_name=$gpo_short_name, \
                            g.gpo_org_type=$gpo_org_type, \
                            g.gpo_address=$gpo_address,\
                            g.country_of_origin=$country_of_origin,\
                            g.source_identifier=$source_identifier,\
                            g.start_date=$start_date,\
                            g.end_date=$end_date,\
                            g.last_update_date=$last_update_date,\
                            g.last_update_user=$last_update_user,\
                            g.version=$version\
              on match set\
                            g.gpo_name=$gpo_name,\
                            g.gpo_short_name=$gpo_short_name, \
                            g.gpo_org_type=$gpo_org_type, \
                            g.gpo_address=$gpo_address,\
                            g.country_of_origin=$country_of_origin,\
                            g.source_identifier=$source_identifier,\
                            g.start_date=$start_date,\
                            g.end_date=$end_date,\
                            g.last_update_date=$last_update_date,\
                            g.last_update_user=$last_update_user,\
                            g.version=$version",
                gpo_id=record['GPO_ID'],gpo_name=record['GPO_NAME'],
                gpo_short_name=record['GPO_SHORT_NAME'],country_of_origin=record['COUNTRY_OF_ORIGIN'],
                gpo_org_type=record['GPO_ORG_TYPE'],gpo_address=['GPO_ADDRESS'],
                source_identifier=record['SOURCE_IDENTIFIER'],start_date=record['START_DATE'],
                end_date=record['END_DATE'],last_update_date=record['LAST_UPDATE_DT'],
                last_update_user=record['LAST_UPDATE_USER'],version=record['VERSION'])

# Creating relationships in Neo4J

In [ ]:
graph.run("match (m: Manufacturer), (p: Product), (c: Correlation_Data) \
           where c.manufacturer_name = m.manufacturer_name and c.product_id = p.product_id \
           merge (p) - [r: created_by] -> (m)")

graph.run("match (m: Manufacturer), (d: Distributor), (c: Correlation_Data) \
           where c.manufacturer_name = m.manufacturer_name and c.distributor_name = d.distributor_name \
           merge (m) - [r: supplied_to] -> (d)")

graph.run("match (h: Hospital), (d: Distributor), (c: Correlation_Data) \
           where h.hospital_name = c.hospital_name and c.distributor_name = d.distributor_name \
           merge (d) - [r: shipped_to] -> (h)")

In [ ]:
graph.run("match (h: Hospital), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and h.hospital_name = c.hospital_name \
           merge (p) - [r: product_shipped_to{hospital_sku: c.hospital_sku}] -> (h)")

graph.run("match (d: Distributor), (p: Product), (c: Correlation_Data) \
           where p.product_id = c.product_id and c.distributor_name = d.distributor_name \
           merge (p) - [r: product_supplied_to{distributor_sku: c.distributor_sku}] -> (d)")

# Queries

In [ ]:
## MANUFACTURER
# who are my distributors:
manufacturer_name='MALLINCKRODT'
print("1. \n",graph.run('match (m:Manufacturer),(d:Distributor),(c:Correlation_Data) \
where c.manufacturer_name=m.manufacturer_name and c.distributor_name=d.distributor_name \
          return distinct m.manufacturer_name, d.distributor_name'))

# who are my dircet customers: 
print("2. \n",graph.run('match (m:Manufacturer),(h:Hospital),(c:Correlation_Data) \
where c.manufacturer_name=m.manufacturer_name and c.hospital_name=h.hospital_name \
          return distinct m.manufacturer_name, h.hospital_name').to_table())

#who are my gpos who contract this sku.
print("3. \n",graph.run('match (c:Correlation_Data), (cn:Contract) \
where c.contract_number = cn.contract_number \
return c.manufacturer_name, c.contract_number, cn.gpo_name, c.product_id').to_table())

#what is contract price and list price for a given gpo for this sku for a given customer.
print("4. \n",graph.run('match (c:Correlation_Data), (cn:Contract) \
where c.contract_number = cn.contract_number \
return c.manufacturer_name, cn.gpo_name, c.hospital_name, c.product_id,\
c.contract_price, c.base_price').to_table())

# show all customers for this GPO for this SKU and the prices
sku= '2001'
gpo = 'GPO_NAME10'
print("5. \n", graph.run('match (c:Correlation_Data), (cn:Contract) \
where c.product_id="'+sku+'" and cn.gpo_name = "'+gpo+'" and c.contract_number=cn.contract_number \
return c.manufacturer_name,c.product_id, cn.gpo_name, c.base_price, c.contract_price').to_table())

In [ ]:
## MANUFACTURER
# who are my distributors:
m_n='Medical Action Industries Inc'
print("1. \n",graph.run('match (m:Manufacturer),(d:Distributor),(c:Correlation_Data) \
where c.manufacturer_name=m.manufacturer_name and c.distributor_name=d.distributor_name and m.manufacturer_name="'+m_n+'"\
          return distinct m.manufacturer_name, d.distributor_name'))

In [ ]:
## Distributor
# what is corresponding sku mapped to customer
print("1. \n",graph.run('match (h:Hospital),(p:Product),(c:Correlation_Data) \
where p.product_id=c.product_id and h.hospital_name=c.hospital_name \
return c.distributor_sku, c.hospital_sku, c.hospital_name').to_table())

# what is corresponding sku mapped to manufacturer
print("2. \n",graph.run('match (d:Distributor),(p:Product),(c:Correlation_Data) \
where p.product_id=c.product_id and c.distributor_name=d.distributor_name \
return c.distributor_sku, p.product_id , p.manufacturer').to_table())

#what is contracted price
print("3. \n",graph.run('match (c:Correlation_Data) \
return c.product_id, c.base_price ,c.contract_price').to_table())

#what are the details of mapped sku
sku= '2001'
print("4. \n",graph.run('match (c:Correlation_Data) , (p:Product) \
where c.product_id = p.product_id and p.product_id ="2001" \
return p , c.distributor_sku'))

# what is contract details for that SKU
print("4. \n",graph.run('match (c:Correlation_Data) , (cn:Contract) \
where c.contract_number = cn.contract_number and c.product_id ="2004" \
return cn'))

In [ ]:
# Hospital/Customer
# what is the mapped distributor sku

print("1. \n",graph.run('match (h:Hospital),(p:Product),(c:Correlation_Data) \
where p.product_id=c.product_id and h.hospital_name=c.hospital_name \
return c.hospital_sku, c.distributor_sku, c.distributor_name').to_table())
 
# what is mapped manufacturer sku
print("2. \n",graph.run('match (m:Manufacturer),(p:Product),(c:Correlation_Data) \
where p.product_id=c.product_id and c.manufacturer_name=m.manufacturer_name \
return  c.hospital_sku, p.product_id , p.manufacturer').to_table())

#what is contracted price for the SKU
print("3. \n",graph.run('match (c:Correlation_Data) \
where c.product_id = "2001"\
return c.product_id, c.base_price ,c.contract_price').to_table())

#what is price and detail on the hospital sku
sku= '8549'
print("4. \n",graph.run('match (c:Correlation_Data) , (p:Product) \
where c.hospital_sku ="5944" and c.product_id = p.product_id \
return p , c.base_price, c.contract_price'))


In [ ]:
# GPO
# what are my contracted customers

print("1. \n",graph.run('match (cn:Contract)  \
where cn.gpo_name="GPO_NAME10" \
return cn.hospital_name'))

# what are the skus contracted thru them: 
print("2. \n",graph.run('match (cn:Contract),(c:Correlation_Data) \
where cn.gpo_name="GPO_NAME10" and cn.contract_number=c.contract_number \
return c.product_id').to_table())

# what is the volume of supply thru GPO
print("3. \n",graph.run('match (cn:Contract),(c:Correlation_Data) \
where cn.gpo_name="GPO_NAME10" and cn.contract_number=c.contract_number \
return count(c)').to_table())



In [ ]:
graph.run("match (n) detach delete n")